In [220]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Input, Flatten

In [221]:
def load_data_csv(filename):
    return pd.read_csv(filename)

In [222]:
def create_image_from_row(X):
    x = X.to_list()
    return_value = np.zeros((28, 28, 1))
    
    data = 0
    for i in range(28):
        for j in range(28):
            return_value[i, j, 0] = x[data] / 255
            data += 1
    
    return return_value

In [223]:
def create_x(df):
    rows = df.shape[0]
    x = []
    for i in range(rows):
        x.append(create_image_from_row(df.iloc[i,:]))
    X = np.array(x)
    return X

In [224]:
def create_y(df):
    y = []
    data = df.to_list()
    
    for i in data:
        new_data = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        
        if i == 0:
            new_data[0] = 1
        elif i == 1:
            new_data[1] = 1
        elif i == 2:
            new_data[2] = 1
        elif i == 3:
            new_data[3] = 1
        elif i == 4:
            new_data[4] = 1
        elif i == 5:
            new_data[5] = 1
        elif i == 6:
            new_data[6] = 1
        elif i == 7:
            new_data[7] = 1
        elif i == 8:
            new_data[8] = 1
        elif i == 9:
            new_data[9] = 1
        y.append(new_data)
        
    return np.array(y)

In [225]:
def split_x_y(df):
    y = df['label']
    x = df.drop(columns='label')
    return x,y

In [226]:
def create_model():
    inputs = Input(shape=(28,28,1))
    x = Conv2D(filters=64, kernel_size=(2,2), strides=(1,1), padding='same', activation='relu')(inputs)
    x = MaxPool2D(pool_size=(2,2), strides=(1,1), padding='same')(x)
    x = Conv2D(filters=64, kernel_size=(2,2), strides=(1,1), padding='same', activation='relu')(x)
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    outputs = Dense(10, activation='softmax')(x)
    
    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer=Adam(lr=0.0001), loss='mse')
    
    return model

In [227]:
def reformat_y(data):
    y = []
    rows = data.shape[0]
    
    for i in range(rows):
        y.append(
            np.argmax(data[i, :])
        )
        
    return np.array(y)

# TRAIN

In [228]:
df = load_data_csv('train.csv')
x, y = split_x_y(df)
X = create_x(x)
Y = create_y(y)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2)

In [229]:
X.shape

(42000, 28, 28, 1)

In [230]:
model = create_model()

In [231]:
model.fit(X_train, y_train, verbose=1, epochs=30, batch_size=32)

Epoch 1/30
1050/1050 [==============================] - 394s 375ms/step - loss: 0.0198
Epoch 2/30
1050/1050 [==============================] - 397s 378ms/step - loss: 0.0037
Epoch 3/30
1050/1050 [==============================] - 380s 362ms/step - loss: 0.0022
Epoch 4/30
1050/1050 [==============================] - 323s 308ms/step - loss: 0.0016
Epoch 5/30
1050/1050 [==============================] - 323s 308ms/step - loss: 0.0011
Epoch 6/30
1050/1050 [==============================] - 322s 307ms/step - loss: 9.1152e-04
Epoch 7/30
1050/1050 [==============================] - 320s 304ms/step - loss: 7.0698e-04
Epoch 8/30
1050/1050 [==============================] - 324s 308ms/step - loss: 5.5126e-04
Epoch 9/30
1050/1050 [==============================] - 327s 311ms/step - loss: 4.3397e-04
Epoch 10/30
1050/1050 [==============================] - 369s 352ms/step - loss: 3.7009e-04
Epoch 11/30
1050/1050 [==============================] - 339s 323ms/step - loss: 3.5326e-04
Epoch 12/30
1050/

In [232]:
y_pred = model.predict(X_test)

In [233]:
orig_y = reformat_y(y_test)
my_y = reformat_y(y_pred)

accuracy_score(orig_y, my_y)

0.9901190476190476

# TEST

In [234]:
df = load_data_csv('test.csv')
X = create_x(df)
y_pred = model.predict(X)
y_pred = reformat_y(y_pred)

In [235]:
# CREATE DICTIONARY FOR DATAFRAME
dataframe_dictionary = {
    'ImageId': range(1, df.shape[0] + 1),
    'Label': y_pred
}

# CREATE DATAFRAME
submission_df = pd.DataFrame(dataframe_dictionary, index=None)
submission_df.to_csv('submission.csv', index=None)